# Contrast and Image Enhancement
## Histogram Equalization
stough 202-
DIP, exactly the content in 3.3.1 on Histogram Equalization.

At this point we've seen the usefulness of transfer functions for contrast enhancement in images, from simple linear windowing to nonliner transforms like power and log. We saw that in this context we look to construct transfer functions that dedicate more of the output range to the subset(s) of the input range where the image intensities are concentrated. In this notebook we'll practically formulate the ideal contrast enhancing transfer function given any particular image.

## Imports

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatLogSlider

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)

from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       vis_pair,
                       lab_uniform)

## Prior Example Transfer Functions

Here is a quick plot and discussion of some of the transfer functions covered in the [enhance_transfer](./enhance_transfer.ipynb) notebook.

In [ ]:
x = np.arange(256)
plt.figure(figsize=(6,4))
plt.plot(x, x, c = 'blue', label = 'equal')
plt.plot(x, np.clip(make_linmap([50,150], [0,255])(x), 0, 255), label = 'win [50,150]')
plt.plot(x, (255/8)*np.log2(1 + x), label = '$log_{2}$')
plt.plot(x, np.power(2,(256/255)*x/32)-1, label = '$2^{x}$')
c = 255.0/(np.power(255.0, 2.4))
plt.plot(x, c*np.power(x,2.4), label = f'$\gamma$ {2.4:.01f}')
plt.plot(x, 16*np.sqrt(x), label = 'sqrt')


plt.xlabel('Input Intensity')
plt.ylabel('Output Intensity')
plt.legend()
plt.tight_layout()

We saw previously that if our image has pixels that are all bunched up in color/intensity, leading to low contrast, our transfer function should accentuate those differences and thus improve contrast. We also saw how the histogram can tell us where to find these concentrations in the input domain.

Seen more practically then, a good transfer function has a <span style="color:#FF9966">**slope above 1, leading to expansion**</span> and higher contrast, where the frequency of intensities is high in the input domain. To balance against that expansion though, the slope must end up <span style="color:#FF9966">**less than 1, leading to compression**</span> of differences and reduction of contrast, in parts of the input domain that are sparsely populated. 

As a result, we saw how an overexposed image, with many bright intensities and few dark ones, could be improved with for example an exponential ($2^{x}$). Similarly an underexposed image, with large dark areas, could be improved by a $log$ or square root transform ($\gamma=1/2$).

### Make Gausisan and Uniform Random Images

See how the uniform has better contrast, see what that means
in the histograms. We can see that higher contrast goes in hand with a uniform histogram.

In [ ]:
I_uniform = np.uint8(256*np.random.random((100,100,3)))
I_gauss = np.clip(128 + 30*np.random.randn(100,100,3), 0, 255).astype('uint8')

In [ ]:
vis_hists(I_uniform)
vis_hists(I_gauss)

### Build a parametric function that would spread the data out in `I_gauss`.
We decided that would be an [S curve or sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function).

$$S(x) = \frac{1}{1 + e^{-x}}$$

But we need a sigmoid that is centered around .5 and steep enough to look like an s inside [0,1]. We can rescale to our $[0,255]$ mapping afterwards

$$S(x, x_c, s) = \frac{1}{1 + e^{-s(x-x_c)}}$$

In [ ]:
def make_scurve(xc, s):
    return lambda x: 1/(1+np.exp(-(s*(x-xc))))

In [ ]:
x = np.arange(0,1,1/100)
# f = lambda x: 1/(1+np.exp(-(10*(x-.5))))
f = make_scurve(.5, 10)
plt.figure(figsize=(4,3))
plt.plot(x, f(x));

### Apply the function to `I_gauss` and see how it improves contrast.

Since the S-curve function will return floating point numbers, for display purposes we must stick to $[0,1]$. 

In [ ]:
vis_pair(I_gauss, f(I_gauss/255.0)) # or (255*f(I_gauss/255.0)).astype('uint8'))

In [ ]:
vis_rgb_cube(f(I_gauss/255.0))

### We can accomplish the same thing emperically, through the [cumulative distribution function](https://en.wikipedia.org/wiki/Cumulative_distribution_function) on the histogram

A fundamental result from probability theory, if $s = \mathbf{T}(r)$ for $\mathbf{T}$ continuous and differentiable:
$$p_s(s) = p_r(r)\begin{vmatrix}{\frac{dr}{ds}}\end{vmatrix}$$

The CDF measures the probability that a random variable $X$ takes on a value less than or equal to $x$:

 $$P(X \le x) = \intop_{0}^{x} p_r(w)dw$$
 
where $p_r$ is the probability distribution (read: histogram) of our random variable $r$. So to transform into a uniform distribution, our transfer function should be:

 $$s = \mathbf{T}(r) = (L-1) \intop_{0}^{r} p_r(w)dw$$
 
Another way of looking at this transfer function is that it <span style="color:#FF9966">transforms an input intensity to that intensity's rank/percentile in the image</span> (then scaled by the output max).

In [ ]:
# First get the histogram of IN, and plot with the cdf.
# normalize so they show up nicely.

bins=np.arange(257)
freq, bins = np.histogram(I_gauss.ravel(), bins=bins)
freq = freq/freq.max()
cdf = np.cumsum(freq)
cdf = cdf/cdf.max()

f = plt.figure(figsize=(4,3))
plt.bar(bins[:-1], freq, width=.5) # width to keep the bars skinny enough.
plt.plot(bins[:-1], cdf, 'r-');

### It's coincidental that the sigmoid is the CDF of a Gaussian.  

Let's apply the function that uses this empirical cdf (coming from the histogram).


In [ ]:
from scipy.interpolate import interp1d

func = interp1d(bins[:-1], cdf, fill_value='extrapolate') 
# 'extrapolate' is important here, otherwise a lot of out of bound errors.

In [ ]:
vis_pair(I_gauss, func(I_gauss))

In [ ]:
# View the two histograms
f, axarr = plt.subplots(1,1, figsize=(6, 3))

funcI = func(I_gauss)

axarr.hist(I_gauss.ravel(), bins=bins, alpha = .6, label = 'Original', color = 'r');
axarr.hist(256*funcI.ravel(), bins=bins, alpha = .6, label = 'Equalized', color = 'g');
axarr.legend(loc = 'upper right');
plt.show()